In [36]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [37]:
train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')


In [38]:
#Conversion using label encoder
#Conversion using label encoder
def clean_data(data, test_data):
    clean_df = data.copy()
    clean_df_test = test_data.copy()

    for f in clean_df.columns:
        if clean_df[f].dtype == 'object':
            label = preprocessing.LabelEncoder()
            label.fit(list(pd.concat([clean_df[f], clean_df_test[f]]).values))
            clean_df[f] = label.transform(list(clean_df[f].values))
            clean_df_test[f] = label.transform(list(clean_df_test[f].values))
    return (clean_df, clean_df_test)

clean_df, clean_df_test = clean_data(train_set, test_set)
clean_df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,34,124840,42041,88.870874,20.0,8,1307,18,19,15,110518,479,9470,32027
1,15,13510,44278,88.041695,28.0,38,582,18,19,15,110518,62,4345,71079
2,15,131784,44278,94.085021,130.0,38,983,18,19,15,110518,62,3564,32440
3,44,86081,39632,89.869797,34.0,53,1014,1,19,15,53573,443,9532,124405
4,44,142349,25325,89.017651,24.0,468,536,3,16,11,95520,522,15765,33649


In [39]:
clean_df_test = clean_df_test.drop(columns=['price', 'index'])

In [40]:
clean_df_test.head()

,country,description,designation,points,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,44,100464,386,95.036469,53,1091,7,19,15,87601,82,14649,0
1,15,32814,17995,90.966405,350,292,18,19,15,110518,596,10714,1
2,44,87100,13908,88.964358,53,1023,1,11,6,12294,72,1925,2
3,44,152214,44278,89.960356,53,1060,13,18,9,61944,130,10716,3
4,44,80531,44278,88.075501,468,300,3,16,11,20301,465,3153,4


In [41]:
y = clean_df.price.values
X = clean_df.drop(['price','id'],axis=1)
X.head()

,country,description,designation,points,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,34,124840,42041,88.870874,8,1307,18,19,15,110518,479,9470
1,15,13510,44278,88.041695,38,582,18,19,15,110518,62,4345
2,15,131784,44278,94.085021,38,983,18,19,15,110518,62,3564
3,44,86081,39632,89.869797,53,1014,1,19,15,53573,443,9532
4,44,142349,25325,89.017651,468,536,3,16,11,95520,522,15765


In [42]:
test_x = clean_df_test.drop(['id'],axis=1)
test_x.head()

,country,description,designation,points,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,44,100464,386,95.036469,53,1091,7,19,15,87601,82,14649
1,15,32814,17995,90.966405,350,292,18,19,15,110518,596,10714
2,44,87100,13908,88.964358,53,1023,1,11,6,12294,72,1925
3,44,152214,44278,89.960356,53,1060,13,18,9,61944,130,10716
4,44,80531,44278,88.075501,468,300,3,16,11,20301,465,3153


In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = X_train.values
test_x = test_x.values

In [51]:
#Fitting XGB regressor 
model = xgb.XGBRegressor(n_estimators=50000, max_depth=10)
model.fit(X_train,y_train)

[09:37:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=50000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [52]:
#sub8 50000 5 yet to run down - 17.6555

In [53]:
from sklearn import metrics
predictions = model.predict(X_test.values)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

RMSE: 21.854051791128736


In [54]:
#Predict 
output = model.predict(test_x)
final_df = pd.DataFrame()
final_df["id"] = clean_df_test['id']
final_df["price"] = output
#final_df.to_csv("Output_1.csv")
final_df = final_df.set_index('id')
final_df.head()

,price
id,
0,110.470131
1,60.667938
2,40.107319
3,28.003935
4,9.132165


In [ ]:
final_df.to_csv('a.csv')